In [1]:
import json
from copy import deepcopy
from collections import defaultdict
from pathlib import Path
from pprint import pprint

from beir.retrieval.evaluation import EvaluateRetrieval
from beir.datasets.data_loader import GenericDataLoader
from common import get_result_colbert, get_result, get_result_bm25, add_result, add_result_org

/home/iida.h/work/IR/JNLP2023-analysis/.venv_202307/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
k_values = [1, 3, 5, 10, 100]

In [3]:
datasets = ["bioask", "nfcorpus", "trec-covid", "scidocs", "scifact"]
retriever = EvaluateRetrieval("", k_values = k_values)

In [4]:
models_name = [["dense-org"], ["splade-org"], ["mlm-splade-tk-org"], ["mlm-splade-tk-idf"], ["dense-org", "splade-org"], ["mlm-dense-org"],  ["mlm-splade-org"], ["mlm-splade-org", "mlm-dense-org"]]
root_path = Path("/home/gaia_data/iida.h/BEIR/datasets")
model_root_dir = Path("/home/gaia_data/iida.h/BEIR/splade_vocab/evaluation")
colbert_root_dir = Path("/home/gaia_data/iida.h/BEIR/colbert/")

In [5]:
def colbert(datasets, root_path, colbert_root_dir, colbert_path_pattern, gpl):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        if gpl:
            colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "Distilcolbert-self", dataset)  
        else:
            colbert_dir = colbert_root_dir.joinpath("experiments", "Distilcolbert-self", dataset)   
        colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
        colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
        colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)

        ndcg, _map, recall, precision = retriever.evaluate(qrels, colbert_result, k_values)
        rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
        all_result[dataset] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "Rcap@100": rcap["R_cap@100"]}
    return all_result

In [6]:
def da_colbert(datasets, root_path, colbert_root_dir, colbert_path_pattern, gpl):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        if dataset in {"bioask", "nfcorpus", "trec-covid"}:
            if gpl:
                colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-bio", dataset)
            else:
                colbert_dir = colbert_root_dir.joinpath("experiments", "DistilAdaLM-bio", dataset)
        elif dataset in {"scidocs", "scifact"}:
            if gpl:
                colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-sci", dataset)
            else:
                colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-sci", dataset)
        colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
        colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
        colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)

        ndcg, _map, recall, precision = retriever.evaluate(qrels, colbert_result, k_values)
        rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
        all_result[dataset] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap@100": rcap["R_cap@100"]}
    return all_result

In [7]:
def add_ensemble_w_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, colbert_path_pattern, gpl):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        for model_name in models_name:
            result = {}
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                if mn == "bm25":
                    model_result = get_result_bm25(model_path)
                else:
                    model_result = get_result(model_path)
                result = add_result(result, model_result)
                if gpl:
                    colbert_dir = colbert_rcoot_dir.joinpath("gpl_experiments", "colbertv2.0", dataset)  
                else:
                    colbert_dir = colbert_root_dir.joinpath("experiments", "colbertv2.0", dataset)
                print(colbert_dir)
                colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
            colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
            colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)
            
            result = add_result(result, colbert_result)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R)cap@100": rcap["R_cap@100"]}
    return all_result

In [8]:
def add_ensemble_w_da_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, colbert_path_pattern, gpl):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        for model_name in models_name:
            result = {}
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                if mn == "bm25":
                    model_result = get_result_bm25(model_path)
                else:
                    model_result = get_result(model_path)
                result = add_result(result, model_result)
                if dataset in {"bioask", "nfcorpus", "trec-covid"}:
                    if gpl:
                        colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-bio", dataset)
                    else:
                        colbert_dir = colbert_root_dir.joinpath("experiments", "DistilAdaLM-bio", dataset)
                elif dataset in {"scidocs", "scifact"}:
                    if gpl:
                        colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-sci", dataset)
                    else:
                        colbert_dir = colbert_root_dir.joinpath("experiments", "DistilAdaLM-sci", dataset)
            if dataset in {"bioask"}:
                print(list(colbert_dir.glob(colbert_path_pattern)))
                colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
            else:    
                colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
            colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
            colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)
            
            result = add_result(result, colbert_result)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap": rcap["R_cap@100"]}
    return all_result

In [9]:
def add_ensemble_w_colbert_and_bm25(datasets, models_name, root_path, model_root_dir, colbert_root_dir):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        bm25_dir = model_root_dir.joinpath(dataset, "bm25")
        bm25_path = list(bm25_dir.glob("**/analysis.json"))[-1]
        bm25_result = get_result_bm25(bm25_path)
        for model_name in models_name:
            # result = deepcopy(bm25_result)
            result = bm25_result
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                model_result = get_result(model_path)
                result = add_result(result, model_result)
            colbert_dir = colbert_root_dir.joinpath("experiments", "colbertv2.0", dataset)   
            colbert_path = list(colbert_dir.glob("*search/2022-12/**/*.tsv"))[0]
            colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
            colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)
            
            result = add_result(result, colbert_result)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap@100": rcap["R_cap@100"]}
    return all_result

In [10]:
def add_ensemble_w_da_colbert_and_bm25(datasets, models_name, root_path, model_root_dir, colbert_root_dir, colbert_path_pattern, gpl):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        bm25_dir = model_root_dir.joinpath(dataset, "bm25")
        bm25_path = list(bm25_dir.glob("**/analysis.json"))[-1]
        bm25_result = get_result_bm25(bm25_path)
        for model_name in models_name:
            result = deepcopy(bm25_result)
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                if mn == "bm25":
                    model_result = get_result_bm25(model_path)
                else:
                    model_result = get_result(model_path)
                result = add_result(result, model_result)
            if dataset in {"bioask", "nfcorpus", "trec-covid"}:
                if gpl:
                    colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-bio", dataset)
                else:
                    colbert_dir = colbert_root_dir.joinpath("experiments", "DistilAdaLM-bio", dataset)
            elif dataset in {"scidocs", "scifact"}:
                if gpl:
                    colbert_dir = colbert_root_dir.joinpath("gpl_experiments", "DistilAdaLM-sci", dataset)
                else:
                    colbert_dir = colbert_root_dir.joinpath("experiments", "DistilAdaLM-sci", dataset)
            if dataset in {"bioask"}:
                print(colbert_dir, list(colbert_dir.glob(colbert_path_pattern)))
                colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
            else:
                colbert_path = list(colbert_dir.glob(colbert_path_pattern))[0]
            
            colbert_dataset_dir = colbert_root_dir.joinpath("datasets", dataset)
            colbert_result = get_result_colbert(colbert_path, colbert_dataset_dir)
            
            result = add_result(result, colbert_result)
            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap@100": rcap["R_cap@100"]}
    return all_result

In [11]:
def add_models_w_bm25(datasets, models_name, root_path, model_root_dir):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        bm25_dir = model_root_dir.joinpath(dataset, "bm25")
        bm25_path = list(bm25_dir.glob("**/analysis.json"))[-1]
        bm25_result = get_result_bm25(bm25_path)
        for model_name in models_name:
            result = deepcopy(bm25_result)
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                print(model_name)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                model_result = get_result(model_path)
                result = add_result(result, model_result)

            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            mns = "-".join(model_name)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "rcap@100": rcap["R_cap@100"]}
    return all_result

In [12]:
def add_models(datasets, models_name, root_path, model_root_dir):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        for model_name in models_name:
            result = {}
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                model_result = get_result(model_path)
                result = add_result(result, model_result)

            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap@100": rcap["R_cap@100"]}
    return all_result

In [15]:
def add_models_cbm25(datasets, models_name, root_path, model_root_dir):
    all_result = defaultdict(dict)
    for dataset in datasets:
        data_path = root_path / dataset
        corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")
        for model_name in models_name:
            result = {}
            for mn in model_name:
                model_dir = model_root_dir.joinpath(dataset, "result", mn)
                model_path = list(model_dir.glob("**/analysis.json"))[-1]
                if "lss" in mn:
                    model_result = get_result(model_path)
                else:
                    modek_result = get_result(model_path)
                result = add_result(result, model_result)

            ndcg, _map, recall, precision = retriever.evaluate(qrels, result, k_values)
            rcap = retriever.evaluate_custom(qrels, result, k_values, metric="r_cap")
            mns = "-".join(model_name)
            all_result[dataset][mns] = {"ndcg@10": ndcg["NDCG@10"], "recall@100": recall["Recall@100"], "R_cap@100": rcap["R_cap@100"]}
    return all_result

In [15]:
# datasets = ["bioask", "nfcorpus", "trec-covid", "scidocs", "scifact"]
datasets = ["bioask"]
models_name = [["mlm-splade-org"],  ["mlm-splade-tk-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:02<00:00, 121810.97it/s]


['mlm-splade-org']
['mlm-splade-tk-org']
defaultdict(<class 'dict'>,
            {'bioask': {'mlm-splade-org': {'ndcg@10': 0.57734,
                                           'rcap@100': 0.82542,
                                           'recall@100': 0.82542},
                        'mlm-splade-tk-org': {'ndcg@10': 0.5511,
                                              'rcap@100': 0.8041,
                                              'recall@100': 0.8041}}})


In [16]:
# datasets = ["bioask", "nfcorpus", "trec-covid", "scidocs", "scifact"]
datasets = ["bioask"]
models_name = [["mlm-splade-idf"],  ["mlm-splade-tk-idf"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [01:53<00:00, 131493.30it/s]


['mlm-splade-idf']
['mlm-splade-tk-idf']
defaultdict(<class 'dict'>,
            {'bioask': {'mlm-splade-idf': {'ndcg@10': 0.57674,
                                           'rcap@100': 0.82077,
                                           'recall@100': 0.82127},
                        'mlm-splade-tk-idf': {'ndcg@10': 0.57292,
                                              'rcap@100': 0.8255,
                                              'recall@100': 0.8255}}})


In [14]:
# datasets = ["bioask", "nfcorpus", "trec-covid", "scidocs", "scifact"]
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact"]
models_name = [["mlm-splade-org"], ["mlm-splade-tk-org"], ["mlm-splade-org", "mlm-dense-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3633/3633 [00:00<00:00, 76263.13it/s]


['mlm-splade-org']
['mlm-splade-tk-org']
['mlm-splade-org', 'mlm-dense-org']
['mlm-splade-org', 'mlm-dense-org']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 96563.26it/s]


['mlm-splade-org']
['mlm-splade-tk-org']
['mlm-splade-org', 'mlm-dense-org']
['mlm-splade-org', 'mlm-dense-org']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25657/25657 [00:01<00:00, 23922.46it/s]


['mlm-splade-org']
['mlm-splade-tk-org']
['mlm-splade-org', 'mlm-dense-org']
['mlm-splade-org', 'mlm-dense-org']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 97261.34it/s]


['mlm-splade-org']
['mlm-splade-tk-org']
['mlm-splade-org', 'mlm-dense-org']
['mlm-splade-org', 'mlm-dense-org']
defaultdict(<class 'dict'>,
            {'nfcorpus': {'mlm-splade-org': {'ndcg@10': 0.3537,
                                             'rcap@100': 0.31059,
                                             'recall@100': 0.2949},
                          'mlm-splade-org-mlm-dense-org': {'ndcg@10': 0.36014,
                                                           'rcap@100': 0.32801,
                                                           'recall@100': 0.31256},
                          'mlm-splade-tk-org': {'ndcg@10': 0.34322,
                                                'rcap@100': 0.29448,
                                                'recall@100': 0.27882}},
             'scidocs': {'mlm-splade-org': {'ndcg@10': 0.16398,
                                            'rcap@100': 0.37052,
                                            'recall@100': 0.37052},
            

In [18]:
# datasets = ["bioask", "nfcorpus", "trec-covid", "scidocs", "scifact"]
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact"]
models_name = [["mlm-splade-idf"], ["mlm-splade-tk-idf"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3633/3633 [00:00<00:00, 86672.58it/s]


['mlm-splade-idf']
['mlm-splade-tk-idf']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 98418.13it/s]


['mlm-splade-idf']
['mlm-splade-tk-idf']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25657/25657 [00:01<00:00, 23703.99it/s]


['mlm-splade-idf']
['mlm-splade-tk-idf']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 99111.78it/s]


['mlm-splade-idf']
['mlm-splade-tk-idf']
defaultdict(<class 'dict'>,
            {'nfcorpus': {'mlm-splade-idf': {'ndcg@10': 0.35647,
                                             'rcap@100': 0.3139,
                                             'recall@100': 0.29799},
                          'mlm-splade-tk-idf': {'ndcg@10': 0.34811,
                                                'rcap@100': 0.29971,
                                                'recall@100': 0.28395}},
             'scidocs': {'mlm-splade-idf': {'ndcg@10': 0.16483,
                                            'rcap@100': 0.37748,
                                            'recall@100': 0.37748},
                         'mlm-splade-tk-idf': {'ndcg@10': 0.16313,
                                               'rcap@100': 0.37082,
                                               'recall@100': 0.37082}},
             'scifact': {'mlm-splade-idf': {'ndcg@10': 0.71765,
                                            'rcap@100'

In [15]:
datasets = ["bioask"]
models_name = [["mlm-splade-org"], ["mlm-splade-org", "mlm-dense-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:20<00:00, 106320.50it/s]


['mlm-splade-org']
['mlm-splade-org', 'mlm-dense-org']
['mlm-splade-org', 'mlm-dense-org']
defaultdict(<class 'dict'>,
            {'bioask': {'mlm-splade-org': {'ndcg@10': 0.57734,
                                           'rcap@100': 0.82542,
                                           'recall@100': 0.82542},
                        'mlm-splade-org-mlm-dense-org': {'ndcg@10': 0.57729,
                                                         'rcap@100': 0.81891,
                                                         'recall@100': 0.81891}}})


In [23]:
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact"]
models_name = [["mlm-dense-org"],  ["mlm-splade-org"], ["mlm-splade-org", "mlm-dense-org"]]
colbert_path_pattern = [("*search/2022-12/29/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_da_colbert_and_bm25(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 96539.60it/s]


defaultdict(<class 'dict'>,
            {'nfcorpus': {'mlm-dense-org': {'R_cap@100': 0.3183,
                                            'ndcg@10': 0.35384,
                                            'recall@100': 0.30384},
                          'mlm-splade-org': {'R_cap@100': 0.32236,
                                             'ndcg@10': 0.35784,
                                             'recall@100': 0.30711},
                          'mlm-splade-org-mlm-dense-org': {'R_cap@100': 0.32522,
                                                           'ndcg@10': 0.35973,
                                                           'recall@100': 0.3104}},
             'scidocs': {'mlm-dense-org': {'R_cap@100': 0.36835,
                                           'ndcg@10': 0.17373,
                                           'recall@100': 0.36835},
                         'mlm-splade-org': {'R_cap@100': 0.38062,
                                            'ndcg@10': 0.16856,
      

In [91]:
datasets = ["bioask"]
models_name = [["mlm-dense-org"],  ["mlm-splade-org"], ["mlm-splade-org", "mlm-dense-org"]]
colbert_path_pattern = [("*search/2023-01/01/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_da_colbert_and_bm25(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

  0%|          | 0/14914714 [00:00<?, ?it/s]

defaultdict(<class 'dict'>,
            {'bioask': {'mlm-dense-org': {'ndcg@10': 0.54182,
                                          'recall@100': 0.7904},
                        'mlm-splade-org': {'ndcg@10': 0.57281,
                                           'recall@100': 0.7904},
                        'mlm-splade-org-mlm-dense-org': {'ndcg@10': 0.54083,
                                                         'recall@100': 0.7904}}})


In [14]:
datasets = ["bioask"]
models_name = [["bm25", "mlm-splade-org"], ["bm25", "mlm-splade-org", "mlm-dense-org"]]
colbert_path_pattern = [("*search/2023-01/01/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_da_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:17<00:00, 108082.67it/s]


[PosixPath('/home/gaia_data/iida.h/BEIR/colbert/experiments/DistilAdaLM-bio/bioask/home.acc12607xq.work.ColBERT.run.scripts.search/2023-01/01/14.14.16/bioask.nbits2.ranking.tsv')]
[PosixPath('/home/gaia_data/iida.h/BEIR/colbert/experiments/DistilAdaLM-bio/bioask/home.acc12607xq.work.ColBERT.run.scripts.search/2023-01/01/14.14.16/bioask.nbits2.ranking.tsv')]
defaultdict(<class 'dict'>,
            {'bioask': {'bm25-mlm-splade-org': {'R_cap': 0.83779,
                                                'ndcg@10': 0.59241,
                                                'recall@100': 0.83779},
                        'bm25-mlm-splade-org-mlm-dense-org': {'R_cap': 0.84146,
                                                              'ndcg@10': 0.57995,
                                                              'recall@100': 0.84146}}})


In [16]:
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact", "bioask"]
models_name = [["lss/mpnet-tod", "dot/mpnet-v3-mse-beir-dot"]]
all_result = add_models_cbm25(datasets, models_name, root_path, Path("/home/gaia_data/iida.h/BEIR/C-BM25/results/"))
pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [01:14<00:00, 200792.96it/s]


defaultdict(<class 'dict'>,
            {'bioask': {'lss/mpnet-tod-dot/mpnet-v3-mse-beir-dot': {'R_cap@100': 0.71936,
                                                                    'ndcg@10': 0.52644,
                                                                    'recall@100': 0.71936}},
             'nfcorpus': {'lss/mpnet-tod-dot/mpnet-v3-mse-beir-dot': {'R_cap@100': 0.25127,
                                                                      'ndcg@10': 0.3607,
                                                                      'recall@100': 0.25164}},
             'scidocs': {'lss/mpnet-tod-dot/mpnet-v3-mse-beir-dot': {'R_cap@100': 0.32987,
                                                                     'ndcg@10': 0.15933,
                                                                     'recall@100': 0.32987}},
             'scifact': {'lss/mpnet-tod-dot/mpnet-v3-mse-beir-dot': {'R_cap@100': 0.917,
                                                              

# No-DA-Ensemble

In [1]:
datasets = ["msmarco"]
models_name = [["dense-org"], ["splade-org"], ["dense-org", "splade-org"]]
all_result = add_models(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

NameError: name 'add_models_w_bm25' is not defined

In [20]:
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact"]
models_name = [["splade-org"], ["dense-org", "splade-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3633/3633 [00:00<00:00, 86434.63it/s]


['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 106342.41it/s]


['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25657/25657 [00:01<00:00, 24229.45it/s]


['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 97301.39it/s]


['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']
defaultdict(<class 'dict'>,
            {'nfcorpus': {'dense-org-splade-org': {'ndcg@10': 0.34493,
                                                   'rcap@100': 0.29709,
                                                   'recall@100': 0.28224},
                          'splade-org': {'ndcg@10': 0.3441,
                                         'rcap@100': 0.30116,
                                         'recall@100': 0.28581}},
             'scidocs': {'dense-org-splade-org': {'ndcg@10': 0.16937,
                                                  'rcap@100': 0.37717,
                                                  'recall@100': 0.37717},
                         'splade-org': {'ndcg@10': 0.16199,
                                        'rcap@100': 0.36867,
                                        'recall@100': 0.36867}},
             'scifact': {'dense-org-splade-org': {'ndcg@10': 0.71234,
                        

In [13]:
datasets = ["msmarco"]
models_name = [["dense-org"], ["splade-org"], ["dense-org", "splade-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8841823/8841823 [00:29<00:00, 302780.09it/s]


['dense-org']
['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']
defaultdict(<class 'dict'>,
            {'msmarco': {'dense-org': {'ndcg@10': 0.71131,
                                       'rcap@100': 0.63959,
                                       'recall@100': 0.55217},
                         'dense-org-splade-org': {'ndcg@10': 0.73818,
                                                  'rcap@100': 0.66757,
                                                  'recall@100': 0.5765},
                         'splade-org': {'ndcg@10': 0.72018,
                                        'rcap@100': 0.66289,
                                        'recall@100': 0.57248}}})


In [14]:
datasets = ["msmarco"]
# models_name = [["bm25", "dense-org", "splade-org"]]
models_name = [["bm25", "splade-org"],["bm25", "dense-org", "splade-org"]]
colbert_path_pattern = [("*search/2022-12/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8841823/8841823 [00:28<00:00, 306011.10it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/msmarco
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/msmarco
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/msmarco
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/msmarco
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/msmarco
defaultdict(<class 'dict'>,
            {'msmarco': {'bm25-dense-org-splade-org': {'R)cap@100': 0.66887,
                                                       'ndcg@10': 0.7469,
                                                       'recall@100': 0.57635},
                         'bm25-splade-org': {'R)cap@100': 0.67695,
                                             'ndcg@10': 0.74788,
                                             'recall@100': 0.58395}}})


In [ ]:
datasets = ["msmarco"]
models_name = [["dense-org"], ["splade-org"], ["dense-org", "splade-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

In [98]:
colbert_path_pattern = [("*search/2023-01/07/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = colbert(datasets, root_path, colbert_root_dir, cpp, gpl)
    pprint(all_result)

  0%|          | 0/8841823 [00:00<?, ?it/s]

defaultdict(<class 'dict'>,
            {'msmarco': {'ndcg@10': 0.74881, 'recall@100': 0.5519}})


In [19]:
datasets = ["nfcorpus", "trec-covid", "scidocs", "scifact"]
# models_name = [["bm25"], ["dense-org"], ["splade-org"], ["dense-org", "splade-org"],["bm25", "dense-org", "splade-org"]]
models_name = [["bm25", "splade-org"], ["bm25", "splade-org", "dense-org"]]
colbert_path_pattern = [("*search/2022-12/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3633/3633 [00:00<00:00, 85791.78it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/nfcorpus
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/nfcorpus
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/nfcorpus
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/nfcorpus
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/nfcorpus


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 171332/171332 [00:01<00:00, 106360.13it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/trec-covid
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/trec-covid
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/trec-covid
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/trec-covid
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/trec-covid


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25657/25657 [00:01<00:00, 24250.04it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scidocs
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scidocs
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scidocs
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scidocs
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scidocs


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5183/5183 [00:00<00:00, 95802.74it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scifact
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scifact
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scifact
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scifact
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/scifact
defaultdict(<class 'dict'>,
            {'nfcorpus': {'bm25-splade-org': {'R)cap@100': 0.30598,
                                              'ndcg@10': 0.34117,
                                              'recall@100': 0.29106},
                          'bm25-splade-org-dense-org': {'R)cap@100': 0.30187,
                                                        'ndcg@10': 0.34114,
                                                        'recall@100': 0.28731}},
             'scidocs': {'bm25-splade-org': {'R)cap@100': 0.3811,
                                             'ndcg@10': 0.16502,
                                             'recall@100': 0.3

In [14]:
datasets = ["bioask"]
models_name = [["splade-org"], ["dense-org", "splade-org"]]
all_result = add_models_w_bm25(datasets, models_name, root_path, model_root_dir)
pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:16<00:00, 109379.74it/s]


['splade-org']
['dense-org', 'splade-org']
['dense-org', 'splade-org']
defaultdict(<class 'dict'>,
            {'bioask': {'dense-org-splade-org': {'ndcg@10': 0.54872,
                                                 'rcap@100': 0.76145,
                                                 'recall@100': 0.76145},
                        'splade-org': {'ndcg@10': 0.57512,
                                       'rcap@100': 0.78845,
                                       'recall@100': 0.78845}}})


In [14]:
datasets = ["bioask"]
models_name = [["bm25", "splade-org"]]
colbert_path_pattern = [("*search/2022-12/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:06<00:00, 118176.81it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/bioask
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/bioask
defaultdict(<class 'dict'>,
            {'bioask': {'bm25-splade-org': {'R)cap@100': 0.79363,
                                            'ndcg@10': 0.57311,
                                            'recall@100': 0.79363}}})


In [13]:
datasets = ["bioask"]
# models_name = [["bm25", "dense-org", "splade-org"]]
models_name = [["bm25", "dense-org", "splade-org"]]
colbert_path_pattern = [("*search/2022-12/**/*.tsv", False)]
for cpp, gpl in colbert_path_pattern:
    all_result = add_ensemble_w_colbert(datasets, models_name, root_path, model_root_dir, colbert_root_dir, cpp, gpl)
    pprint(all_result)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14914714/14914714 [02:11<00:00, 113510.31it/s]


/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/bioask
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/bioask
/home/gaia_data/iida.h/BEIR/colbert/experiments/colbertv2.0/bioask
defaultdict(<class 'dict'>,
            {'bioask': {'bm25-dense-org-splade-org': {'R)cap@100': 0.81969,
                                                      'ndcg@10': 0.57043,
                                                      'recall@100': 0.81969}}})
